This code is from the Boden Miller group and takes a zip of IMC output files (MCD+txt) and creates an analysis folder which contains multichannel TIFFs. References to ilastik have beebn removed. See IMCrun2 folder for earlier versions with ilastik output code.

In [ ]:


from imctools.converters import ome2analysis
from imctools.converters import ome2histocat
from imctools.converters import mcdfolder2imcfolder
from imctools.converters import exportacquisitioncsv



In [ ]:
## This will output the directory you are in amoungst other things. The first output is what you should use in the 4th cell for 'folders_path_inputs'.
import sys
print(sys.path)
print(sys.executable)


In [ ]:


import os
import pathlib
import shutil
import re



The below window is where you set the directory location for your input zip. You need a panel CSV (see example layout of panel csv) too. Note the double backslashes.

Once you have entered the correct file locations in the box below, everythign else can just be run.

In [ ]:
# the input_data_folders_path_inputs with the zipped acquisition files for the analysis
folders_path_inputs = ['C:\\Users\\nbw33\\OneDrive - Newcastle University\\IMC\\IMCtest'] ## Set this

# part that all considered files need to have in common (no need to change this)
input_file_regexp = '.*.zip'

# output for OME tiffs
folder_path_base = 'C:\\Users\\nbw33\\OneDrive - Newcastle University\\IMC\\IMCtest\\analysis' ## Set this (it will create this output folder if new)


# pannel
file_path_csv_panel = 'C:\\Users\\nbw33\\OneDrive - Newcastle University\\IMC\\IMCtest\\Paneltest.csv'  ## Set this
csv_panel_metal = 'Metal Tag'
csv_panel_full = 'full'

In [ ]:
folder_path_base = pathlib.Path(folder_path_base)
folders_path_inputs = [pathlib.Path(f) for f in folders_path_inputs]
# parameters for resizing the images for ilastik

folder_path_analysis = folder_path_base / 'tiffs'
folder_path_ome= folder_path_base / 'ometiff'
folder_path_cp = folder_path_base / 'cpout'
folder_path_cp_input = folder_path_base / 'cpinp'


# Other output
file_path_cp_csv = folder_path_cp / 'panel.csv'
file_path_full_channels_csv = folder_path_cp_input / 'full_channelmeta.csv'
file_path_prob_channels_csv = folder_path_cp_input / 'probab_channelmeta_manual.csv'

suffix_full = '_full'
suffix_ilastik_scale = '_s2'
suffix_mask = '_mask.tiff'
suffix_probablities = '_Probabilities'


failed_images = list()

In [ ]:


for fol in [folder_path_base, folder_path_analysis,
            folder_path_ome, folder_path_cp,
           folder_path_cp_input]:
    if not fol.exists():
        fol.mkdir(parents=True)



In [ ]:


%%time
failed_images = list()
re_fn = re.compile(input_file_regexp)

for fol in folders_path_inputs:
    for fn in fol.glob('*'):
        if re_fn.match(fn.name):
            mcdfolder2imcfolder.mcdfolder_to_imcfolder(fn, output_folder=folder_path_ome,
                                                       create_zip=False)



In [ ]:
exportacquisitioncsv.export_acquisition_csv(folder_path_ome, output_folder=folder_path_cp_input)

In [ ]:
shutil.copy(file_path_csv_panel, file_path_cp_csv)

In [ ]:
list_analysis_stacks =[
    (csv_panel_full, suffix_full, 0)]

In [ ]:
%%time
ome2analysis.omefolder_to_analysisfolder(folder_path_ome, folder_path_analysis, panel_csv_file=file_path_csv_panel,
                                         analysis_stacks=(list_analysis_stacks), metalcolumn=csv_panel_metal)